<a href="https://colab.research.google.com/github/Krumpu/Homework_ICA/blob/main/HW_1_ICA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('_mpl-gallery')

In [6]:
url = 'https://raw.githubusercontent.com/Krumpu/Homework_ICA/main/Data-Melbourne_F.csv'
df = pd.read_csv(url)

# We can remove "year", "month" and "day" columns,
# since we did no time series analysis today...
# "VG" is removed, we do not have any info about
df = df.drop(['VG', 'year', 'month', 'day'], axis=1)

# Information about our data set
print('Number of Observations (N): ', df.shape[0])
print('Number of Predictor Variables (D): ', df.shape[1])

df.head()

print(df["BOD"].describe())
print(df["COD"].describe())


Number of Observations (N):  1382
Number of Predictor Variables (D):  16
count    1382.000000
mean      382.061708
std        85.996012
min       140.000000
25%       330.000000
50%       360.000000
75%       422.980000
max       850.000000
Name: BOD, dtype: float64
count    1382.000000
mean      845.960434
std       145.416540
min       360.000000
25%       751.250000
50%       845.000000
75%       920.000000
max      1700.000000
Name: COD, dtype: float64


#**Classes**

Para melhor esclarecimento iremos usar duas variaveis como indicadores de economia/deseperdicio durante o tratamento. Sendo elas COD e BOD sendo respectivamente Chemical Oxygen Demand (mg/L) e Biological Oxygen Demand (mg/L). para nossas classes podemos divir o intervalo desses indicadores em n partes, e usar essas partes para predizer a saída. \
Chemical Oxygen Demand(COD), indica a quantidade de oxigênio utilizado durante a reação quimica para consumir contaminantes presentes. \
Biological Oxygen Demand(BOD), é a unidade de medida utilizada para determinar a quantidade de oxigênio utilizado por organismos biológicos aeróbicos.\
Tendo esses dois indicativos iremos forma 4 classes, sendo elas as combinações dos dois intervalos medios de COD E BOD.


In [ ]:
uncond_analysis = pd.DataFrame(columns=['name', 'mean', 'standard_deviation', 'skewness'])

for i, predictor in enumerate(df.columns):
    uncond_analysis.loc[i] = [predictor, df[predictor].mean(), df[predictor].std(), df[predictor].skew()]

    fig, axs = plt.subplots(1, 2, figsize=(16,8))

    _, bins, _ = axs[0].hist(df[predictor], bins=16, linewidth=0.5, edgecolor='white')
    axs[0].set(xticks=bins[::2], title=f'{predictor} histogram')
    axs[1].boxplot(df[predictor], patch_artist=True,
                medianprops={"color": "white", "linewidth": 0.5},
                boxprops={"facecolor": "C0", "edgecolor": "white",
                          "linewidth": 0.5},
                whiskerprops={"color": "C0", "linewidth": 1.5},
                capprops={"color": "C0", "linewidth": 1.5})
    axs[1].set(title=f'{predictor} boxplot')

# For index starting at 1
uncond_analysis.index = uncond_analysis.index + 1
uncond_analysis